In [ ]:
!pip install flask flask-socketio pyngrok
!pip install flask flask-socketio eventlet
!pip install pyngrok

In [1]:
from flask import Flask, render_template_string
from flask_socketio import SocketIO, emit

app = Flask(__name__)
socketio = SocketIO(app)

# HTML Template with embedded CSS and JavaScript
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Video Call</title>
    <style>
        .video-container {
            display: flex;
            justify-content: space-around;
            align-items: center;
            height: 80vh;
        }

        video {
            width: 45%;
            border-radius: 10px;
            background-color: black;
        }

        button {
            display: block;
            margin: 20px auto;
            padding: 10px 20px;
            font-size: 16px;
        }
    </style>
</head>
<body>
    <div class="video-container">
        <video id="localVideo" autoplay muted></video>
        <video id="remoteVideo" autoplay></video>
    </div>
    <button id="startCall">Start Call</button>
    <button id="endCall">End Call</button>
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>
    <script>
        const localVideo = document.getElementById('localVideo');
        const remoteVideo = document.getElementById('remoteVideo');
        const startCall = document.getElementById('startCall');
        const endCall = document.getElementById('endCall');

        const socket = io.connect();

        let localStream;
        let peerConnection;
        const configuration = {
            'iceServers': [{'urls': 'stun:stun.l.google.com:19302'}]
        };

        startCall.onclick = async () => {
            localStream = await navigator.mediaDevices.getUserMedia({video: true, audio: true});
            localVideo.srcObject = localStream;

            peerConnection = new RTCPeerConnection(configuration);
            peerConnection.addStream(localStream);

            peerConnection.ontrack = (event) => {
                remoteVideo.srcObject = event.streams[0];
            };

            peerConnection.onicecandidate = (event) => {
                if (event.candidate) {
                    socket.emit('message', {'candidate': event.candidate});
                }
            };

            const offer = await peerConnection.createOffer();
            await peerConnection.setLocalDescription(offer);
            socket.emit('message', {'sdp': peerConnection.localDescription});
        };

        endCall.onclick = () => {
            peerConnection.close();
            localStream.getTracks().forEach(track => track.stop());
        };

        socket.on('message', async (message) => {
            if (message.sdp) {
                await peerConnection.setRemoteDescription(new RTCSessionDescription(message.sdp));
                if (message.sdp.type === 'offer') {
                    const answer = await peerConnection.createAnswer();
                    await peerConnection.setLocalDescription(answer);
                    socket.emit('message', {'sdp': peerConnection.localDescription});
                }
            } else if (message.candidate) {
                await peerConnection.addIceCandidate(new RTCIceCandidate(message.candidate));
            }
        });
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(template)

@socketio.on('message')
def handle_message(message):
    emit('message', message, broadcast=True)

if __name__ == '__main__':
    socketio.run(app, host='0.0.0.0', port=10000, debug=True)



 * Restarting with stat
Traceback (most recent call last):
  File "/usr/local/python/3.10.13/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/python/3.10.13/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/workspaces/Private-Chat-app/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/workspaces/Private-Chat-app/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/workspaces/Private-Chat-app/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/workspaces/Private-Chat-app/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/workspaces/Private-Chat-app/.venv/lib/python3.10/site-packages/i

SystemExit: 1

/workspaces/Private-Chat-app/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!ngrok authtoken 2gx07k2dnZkMR8dqD4nxLvyOmLJ_6ZUVQ6exmRovj113uQJXz
!ngrok restart

In [2]:
import os
from flask import Flask, request, send_from_directory, render_template_string
from flask_socketio import SocketIO, emit
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

public_chat_history = []
private_messages = {}
users = {}
user_order = []

# Function to delete all files in the uploads folder
def clear_uploaded_images():
    for filename in os.listdir(app.config['UPLOAD_FOLDER']):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        os.remove(file_path)

# HTML template for the chat application
chat_html = '''<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Chakra+Petch:wght@300;400;500;600;700&display=swap" rel="stylesheet">
  <link href="https://unpkg.com/@bentoproject/core@0.2.3/dist/styles.css" rel="stylesheet">
  <title>Spider-Chat</title>
  <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
  <style>
    body {
      font-family: 'Chakra Petch', sans-serif;
      margin: 0;
      display: flex;
      flex-direction: column;
      height: 100vh;
      background-color: #fef7dadc;
      color: #ffffff;
    }
    .chat-container {
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 10px;
    }
    #header {
      color: white;
      text-align: center;
      padding: 10px 0;
      font-size: 24px;
      position:sticky;
      top: 46px;
      z-index: 10000;
    }
    #hamburger {
      position: absolute;
      top: 50%;
      left: 10px;
      transform: translateY(-50%);
      cursor: pointer;
      z-index: 10000;
      width: 50px;
      height: 50px;
      display: flex;
      align-items: center;
      justify-content: center;
      border-radius: 50%;
      background: black;
      transition: all 0.3s ease;
    }
    .line {
      position: absolute;
      height: 2px;
      background-color: white;
      transition: all 0.3s ease;
    }
    .line1 {
      width: 20px;
      top: 40%;
    }
    .line2 {
      width: 10px;
      top: 60%;
    }
    #hamburger.open .line1 {
      transform: rotate(45deg);
      width: 20px;
      top: 50%;
    }
    #hamburger.open .line2 {
      transform: rotate(-45deg);
      width: 20px;
      top: 50%;
    }
    #messages {
      flex: 1;
      overflow-y: auto;
      margin: 10px 0;
    }
    .input-container {
      display: flex;
      padding: 10px 0;
      position:sticky;
      bottom:15px;
      background-color:#fef7da;
    }
    #myMessage {
      flex: 1;
      padding: 10px;
      border: 1px solid #333;
      border-radius: 40px;
      margin-right: 10px;
      background: #1e1e1e;
      color: #fff;
    }
    #sendbutton {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
    }
    #attachment {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
      margin-right: 10px;
    }
    .navbar {
      background-color: #000000;
      border-left: 1px solid #333;
      position: fixed;
      bottom: -100%;
      width: 100%;
      height: 82%;
      transition: bottom ease-in-out 0.3s;
      z-index: 1000;
      border-radius: 50px;
      border-bottom-left-radius: 0;
      border-bottom-right-radius: 0;
      color: #fef7da;
    }
    .navbar.open {
      bottom: 0;
    }
    h2{
      color: #fef7da;
    }
    .navbar h2 {
      text-align: center;
      padding: 10px 0;
      margin: 10px;
      z-index: 1000;
      text-align: center;
      padding: 20px;
      margin-top: 20px;
      border:3px solid #fef7da;
      border-radius: 50px;
      color: #fef7da;
      font-size: 22px;
      font-weight: 700;
    }
    #userList {
      list-style: none;
      padding: 0;
      margin: 0;
    }
    #userList li {
      padding: 10px;
      cursor: pointer;
      border-bottom: 1px solid #333;
      color: #fff;
    }
    .message {
      display: flex;
      align-items: center;
      margin-bottom: 10px;
    }
    .message .profile-pic {
      width: 40px;
      height: 40px;
      margin-right: 10px;
    }
    .message .profile-pic img {
      width: 100%;
      border-radius: 50%;
    }
    #typing-indicator {
      color: #000;
      font-style: italic;
      padding: 0 10px;
      z-index: 1000;
      margin-top: -20px;
    }
    @media (max-width: 768px) {
      .navbar {
        width: 100%;
      }
      #header {
        font-size: 28px;
      }
      #myMessage {
        font-size: 18px;
      }
      #sendbutton {
        font-size: 18px;
      }
      #userList li {
        font-size: 18px;
      }
      .message {
        font-size: 18px;
      }
    }
    .texter {
      text-align: center;
      padding: 20px;
      margin-top: -10px;
      border:3px solid #000000;
      border-radius: 50px;
      color: #000;
      font-size: 22px;
      font-weight: 700;
      position:sticky;
      top:20px;
      background-color: #fef7da;
    }
    div {
      color:#000;
    }
  </style>
</head>
<body>
  <div class="chat-container">
    <div id="header">
      <div id="hamburger">
        <div class="line line1"></div>
        <div class="line line2"></div>
      </div>
    </div>
    <div class="texter">
      Spider-Chat
    </div>
    <div id="messages"></div>
    <div id="typing-indicator"></div>
    <div class="input-container">
      <button id="attachment"><i class="fas fa-paperclip"></i></button>
      <input id="myMessage" placeholder="Type here" autocomplete="off">
      <button id="sendbutton"><i class="fas fa-paper-plane"></i></button>
      <input type="file" id="fileInput" style="display: none;" accept="image/*">
    </div>
  </div>
  <div class="navbar" id="navbar">
    <h2>Users</h2>
    <ul id="userList"></ul>
  </div>
  <script type="text/javascript">
    document.addEventListener('DOMContentLoaded', (event) => {
      var socket = io.connect(window.location.href);
      var navbar = document.getElementById('navbar');
      var hamburger = document.getElementById('hamburger');
      var messageInput = document.getElementById('myMessage');
      var typingIndicator = document.getElementById('typing-indicator');
      var typingTimeout;

      function sendMessage(message, to) {
        if (message.trim() !== '') {
          const data = { message: message.trim(), to: to };
          socket.emit('send_message', data);
        }
      }

      function sendImage(file, to) {
        const formData = new FormData();
        formData.append('image', file);
        formData.append('to', to);
        fetch('/upload_image', {
          method: 'POST',
          body: formData
        }).then(response => response.json())
        .then(data => {
          if (data.url) {
            socket.emit('send_image', { url: data.url, to: to });
          }
        });
      }

      function renderMessage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageText = document.createElement('div');
        messageText.innerText = message.username + ': ' + message.text;
        messageElement.appendChild(messageText);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      function renderImage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageImage = document.createElement('div');
        messageImage.innerHTML = '<img src="' + message.url + '" style="max-width: 200px; border-radius: 10px;">';
        messageElement.appendChild(messageImage);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      socket.on('connect', () => {
        const username = prompt('Enter your username:');
        if (username) {
          socket.emit('set_username', { username: username });
        }
      });

      socket.on('user_list', (user_list) => {
        const userListElement = document.getElementById('userList');
        userListElement.innerHTML = '';
        user_list.forEach((user) => {
          const userElement = document.createElement('li');
          userElement.innerText = user;
          userElement.addEventListener('click', () => {
            const message = prompt('Enter your private message:');
            if (message) {
              sendMessage(message, user);
            }
          });
          userListElement.appendChild(userElement);
        });
      });

      socket.on('receive_message', (data) => {
        renderMessage(data);
      });

      socket.on('receive_image', (data) => {
        renderImage(data);
      });

      document.getElementById('sendbutton').addEventListener('click', () => {
        sendMessage(messageInput.value);
        messageInput.value = '';
        socket.emit('stop_typing');
      });

      document.getElementById('attachment').addEventListener('click', () => {
        document.getElementById('fileInput').click();
      });

      document.getElementById('fileInput').addEventListener('change', (event) => {
        const file = event.target.files[0];
        if (file) {
          sendImage(file);
        }
      });

      hamburger.addEventListener('click', () => {
        navbar.classList.toggle('open');
        hamburger.classList.toggle('open');
      });

      socket.on('chat_history', (history) => {
        history.forEach((message) => {
          if (message.type === 'text') {
            renderMessage(message);
          } else if (message.type === 'image') {
            renderImage(message);
          }
        });
      });

      // Send message on Enter key press
      messageInput.addEventListener('keydown', (event) => {
        if (event.key === 'Enter') {
          sendMessage(messageInput.value);
          messageInput.value = '';
          socket.emit('stop_typing');
        } else {
          socket.emit('typing');
          clearTimeout(typingTimeout);
          typingTimeout = setTimeout(() => {
            socket.emit('stop_typing');
          }, 1000);
        }
      });

      // Notify server on page unload
      window.addEventListener('beforeunload', () => {
        socket.emit('disconnect');
      });

      socket.on('typing', (data) => {
        typingIndicator.innerText = data.username + ' is typing...';
        typingIndicator.style.display = 'block';
      });

      socket.on('stop_typing', () => {
        typingIndicator.style.display = 'none';
      });
    });
  </script>
</body>
</html>'''

@app.route('/')
def index():
    return render_template_string(chat_html)

@app.route('/upload_image', methods=['POST'])
def upload_image():
    file = request.files['image']
    if file:
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        return {'url': f'/uploads/{filename}'}
    return {'url': None}

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

@socketio.on('send_message')
def handle_send_message(data):
    username = users.get(request.sid, 'Unknown')
    message = {'type': 'text', 'username': username, 'text': data['message']}
    public_chat_history.append(message)
    emit('receive_message', message, broadcast=True)

@socketio.on('send_image')
def handle_send_image(data):
    username = users.get(request.sid, 'Unknown')
    message = {'type': 'image', 'username': username, 'url': data['url']}
    public_chat_history.append(message)
    emit('receive_image', message, broadcast=True)

@socketio.on('set_username')
def handle_set_username(data):
    users[request.sid] = data['username']
    user_order.append(data['username'])
    emit('user_list', user_order, broadcast=True)
    emit('chat_history', public_chat_history)

@socketio.on('typing')
def handle_typing():
    username = users.get(request.sid, 'Unknown')
    emit('typing', {'username': username}, broadcast=True, include_self=False)

@socketio.on('stop_typing')
def handle_stop_typing():
    emit('stop_typing', broadcast=True, include_self=False)

@socketio.on('disconnect')
def handle_disconnect():
    if request.sid in users:
        del users[request.sid]
    emit('user_list', list(users.values()), broadcast=True)

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(" * Tunnel URL:", public_url)
    socketio.run(app, port=5000)

 * Tunnel URL: NgrokTunnel: "https://7aac-4-240-39-192.ngrok-free.app" -> "http://localhost:5000"


t=2024-08-26T12:13:42+0000 lvl=warn msg="failed to open private leg" id=59f45c48aec1 privaddr=localhost:5000 err="dial tcp [::1]:5000: connect: connection refused"
t=2024-08-26T12:13:46+0000 lvl=warn msg="failed to open private leg" id=b501bad1ef08 privaddr=localhost:5000 err="dial tcp [::1]:5000: connect: connection refused"
t=2024-08-26T12:13:53+0000 lvl=warn msg="failed to open private leg" id=86cbe4123e28 privaddr=localhost:5000 err="dial tcp [::1]:5000: connect: connection refused"


In [1]:
import os
from flask import Flask, request, send_from_directory, render_template_string
from flask_socketio import SocketIO, emit
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import pygame

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

public_chat_history = []
private_messages = {}
users = {}
user_order = []

# Initialize pygame mixer for playing sound effects
pygame.mixer.init()

# Load sound files
bye_bye_sound = '/mnt/data/bye-bye-lumi-athena-sfx.mp3'
discord_notification_sound = '/mnt/data/discord-notification.mp3'

# Function to delete all files in the uploads folder
def clear_uploaded_images():
    for filename in os.listdir(app.config['UPLOAD_FOLDER']):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        os.remove(file_path)

# HTML template for the chat application
chat_html = '''<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Chakra+Petch:wght@300;400;500;600;700&display=swap" rel="stylesheet">
  <link href="https://unpkg.com/@bentoproject/core@0.2.3/dist/styles.css" rel="stylesheet">
  <title>Spider-Chat</title>
  <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
  <style>
    body {
      font-family: 'Chakra Petch', sans-serif;
      margin: 0;
      display: flex;
      flex-direction: column;
      height: 100vh;
      background-color: #fef7dadc;
      color: #ffffff;
    }
    .chat-container {
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 10px;
    }
    #header {
      color: white;
      text-align: center;
      padding: 10px 0;
      font-size: 24px;
      position:sticky;
      top: 46px;
      z-index: 10000;
    }
    #hamburger {
      position: absolute;
      top: 50%;
      left: 10px;
      transform: translateY(-50%);
      cursor: pointer;
      z-index: 10000;
      width: 50px;
      height: 50px;
      display: flex;
      align-items: center;
      justify-content: center;
      border-radius: 50%;
      background: black;
      transition: all 0.3s ease;
    }
    .line {
      position: absolute;
      height: 2px;
      background-color: white;
      transition: all 0.3s ease;
    }
    .line1 {
      width: 20px;
      top: 40%;
    }
    .line2 {
      width: 10px;
      top: 60%;
    }
    #hamburger.open .line1 {
      transform: rotate(45deg);
      width: 20px;
      top: 50%;
    }
    #hamburger.open .line2 {
      transform: rotate(-45deg);
      width: 20px;
      top: 50%;
    }
    #messages {
      flex: 1;
      overflow-y: auto;
      margin: 10px 0;
    }
    .input-container {
      display: flex;
      padding: 10px 0;
      position:sticky;
      bottom:15px;
      background-color:#fef7da;
    }
    #myMessage {
      flex: 1;
      padding: 10px;
      border: 1px solid #333;
      border-radius: 40px;
      margin-right: 10px;
      background: #1e1e1e;
      color: #fff;
    }
    #sendbutton {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
    }
    #attachment {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
      margin-right: 10px;
    }
    .navbar {
      background-color: #000000;
      border-left: 1px solid #333;
      position: fixed;
      bottom: -100%;
      width: 100%;
      height: 82%;
      transition: bottom ease-in-out 0.3s;
      z-index: 1000;
      border-radius: 50px;
      border-bottom-left-radius: 0;
      border-bottom-right-radius: 0;
      color: #fef7da;
    }
    .navbar.open {
      bottom: 0;
    }
    h2{
      color: #fef7da;
    }
    .navbar h2 {
      text-align: center;
      padding: 10px 0;
      margin: 10px;
      z-index: 1000;
      text-align: center;
      padding: 20px;
      margin-top: 20px;
      border:3px solid #fef7da;
      border-radius: 50px;
      color: #fef7da;
      font-size: 22px;
      font-weight: 700;
    }
    #userList {
      list-style: none;
      padding: 0;
      margin: 0;
    }
    #userList li {
      padding: 10px;
      cursor: pointer;
      border-bottom: 1px solid #333;
      color: #fff;
    }
    .message {
      display: flex;
      align-items: center;
      margin-bottom: 10px;
    }
    .message .profile-pic {
      width: 40px;
      height: 40px;
      margin-right: 10px;
    }
    .message .profile-pic img {
      width: 100%;
      border-radius: 50%;
    }
    #typing-indicator {
      color: #000;
      font-style: italic;
      padding: 0 10px;
      z-index: 1000;
      margin-top: -20px;
    }
    @media (max-width: 768px) {
      .navbar {
        width: 100%;
      }
      #header {
        font-size: 28px;
      }
      #myMessage {
        font-size: 18px;
      }
      #sendbutton {
        font-size: 18px;
      }
      #userList li {
        font-size: 18px;
      }
      .message {
        font-size: 18px;
      }
    }
    .texter {
      text-align: center;
      padding: 20px;
      margin-top: -10px;
      border:3px solid #000000;
      border-radius: 50px;
      color: #000;
      font-size: 22px;
      font-weight: 700;
      position:sticky;
      top:20px;
      background-color: #fef7da;
    }
    div {
      color:#000;
    }
  </style>
</head>
<body>
  <div class="chat-container">
    <div id="header">
      <div id="hamburger">
        <div class="line line1"></div>
        <div class="line line2"></div>
      </div>
    </div>
    <div class="texter">
      Spider-Chat
    </div>
    <div id="messages"></div>
    <div id="typing-indicator"></div>
    <div class="input-container">
      <button id="attachment"><i class="fas fa-paperclip"></i></button>
      <input id="myMessage" placeholder="Type here" autocomplete="off">
      <button id="sendbutton"><i class="fas fa-paper-plane"></i></button>
      <input type="file" id="fileInput" style="display: none;" accept="image/*">
    </div>
  </div>
  <div class="navbar" id="navbar">
    <h2>Users</h2>
    <ul id="userList"></ul>
  </div>
  <script type="text/javascript">
    document.addEventListener('DOMContentLoaded', (event) => {
      var socket = io.connect(window.location.href);
      var navbar = document.getElementById('navbar');
      var hamburger = document.getElementById('hamburger');
      var messageInput = document.getElementById('myMessage');
      var typingIndicator = document.getElementById('typing-indicator');
      var userTyping = false;
      var timeout;

      function timeoutFunction() {
        userTyping = false;
        socket.emit('typing', { typing: false });
      }

      messageInput.addEventListener('keypress', function(e) {
        if (e.which !== 13) {
          if (userTyping === false) {
            userTyping = true;
            socket.emit('typing', { typing: true });
            clearTimeout(timeout);
            timeout = setTimeout(timeoutFunction, 2000);
          } else {
            clearTimeout(timeout);
            timeout = setTimeout(timeoutFunction, 2000);
          }
        }
      });

      messageInput.addEventListener("keyup", function(event) {
        if (event.key === "Enter") {
          var message = messageInput.value;
          socket.emit('message', { message: message });
          messageInput.value = '';
          userTyping = false;
          socket.emit('typing', { typing: false });
          clearTimeout(timeout);
        }
      });

      hamburger.addEventListener('click', function() {
        this.classList.toggle('open');
        navbar.classList.toggle('open');
      });

      socket.on('connect', function() {
        var username = prompt("Please enter your name:");
        socket.emit('add_user', { username: username });
      });

      socket.on('update_user_list', function(data) {
        var userList = document.getElementById('userList');
        userList.innerHTML = '';
        data.users.forEach(function(user) {
          var li = document.createElement('li');
          li.textContent = user;
          userList.appendChild(li);
        });
      });

      socket.on('message', function(data) {
        var messagesDiv = document.getElementById('messages');
        var messageElement = document.createElement('div');
        messageElement.className = 'message';
        messageElement.innerHTML = '<div class="profile-pic"><img src="' + data.avatar + '" alt="Profile"></div><div><strong>' + data.username + ':</strong> ' + data.message + '</div>';
        messagesDiv.appendChild(messageElement);
        messagesDiv.scrollTop = messagesDiv.scrollHeight;

        // Play the bye-bye sound if the message is "bye" or "Bye"
        if (data.message.trim().toLowerCase() === "bye") {
          playSound("bye_bye");
        } else {
          playSound("discord_notification");
        }
      });

      socket.on('typing', function(data) {
        if (data.typing) {
          typingIndicator.innerHTML = data.username + " is typing...";
        } else {
          typingIndicator.innerHTML = '';
        }
      });

      document.getElementById('attachment').addEventListener('click', function() {
        document.getElementById('fileInput').click();
      });

      document.getElementById('fileInput').addEventListener('change', function() {
        var file = this.files[0];
        var formData = new FormData();
        formData.append('file', file);
        fetch('/upload', {
          method: 'POST',
          body: formData
        })
        .then(response => response.json())
        .then(data => {
          socket.emit('message', { message: "Uploaded: " + data.filename });
        })
        .catch(error => {
          console.error('Error uploading file:', error);
        });
      });

      function playSound(sound) {
        if (sound === "bye_bye") {
          var audio = new Audio('/static/bye-bye-lumi-athena-sfx.mp3');
        } else if (sound === "discord_notification") {
          var audio = new Audio('/static/discord-notification.mp3');
        }
        audio.play();
      }
    });
  </script>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(chat_html)

@app.route('/upload', methods=['POST'])
def upload():
    file = request.files['file']
    if file:
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        return {"filename": filename}
    return {"error": "File not uploaded"}

@socketio.on('add_user')
def handle_add_user(data):
    username = data['username']
    if username not in users:
        users[username] = {'messages': []}
        user_order.append(username)
    emit('update_user_list', {'users': user_order}, broadcast=True)

@socketio.on('message')
def handle_message(data):
    message = data['message']
    username = request.sid
    public_chat_history.append({'username': username, 'message': message})
    emit('message', {'username': username, 'message': message, 'avatar': 'https://www.gravatar.com/avatar/' + username + '?d=identicon'}, broadcast=True)

@socketio.on('typing')
def handle_typing(data):
    username = request.sid
    emit('typing', {'username': username, 'typing': data['typing']}, broadcast=True, include_self=False)

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(" * Tunnel URL:", public_url)
    socketio.run(app, port=5000)


pygame 2.6.0 (SDL 2.28.4, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default


error: ALSA: Couldn't open audio device: No such file or directory

In [5]:
!pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 36.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
